In [ ]:
import pystac
from pathlib import Path
from shapely.geometry import Polygon, mapping

import pandas as pd
from rasterio.warp import transform_bounds, transform_geom

import geopandas as gpd
from datetime import datetime, timezone


In [ ]:
catalog = pystac.Catalog(
    id='forecast-test',
    title="Landsat FOVs",
    description='This is a test of Landsat image forecasts',
    stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json']
)

In [ ]:
grid = gpd.read_file('./tmp/all_revisits.geojson')
bbox = grid.total_bounds
print(grid.crs.to_string())
print(bbox)

In [ ]:
def get_bbox_and_footprint(bbox):

    # create the footprint
    footprint = Polygon([
        [bbox[0], bbox[1]],
        [bbox[0], bbox[3]],
        [bbox[2], bbox[3]],
        [bbox[2], bbox[1]]
    ])

    return bbox, mapping(footprint)

In [ ]:
get_bbox_and_footprint(grid.total_bounds)

In [ ]:
folder = Path('./tmp/')
file_list = list(folder.rglob('*fovs.geojson'))

fovs = gpd.read_file(file_list[0])

fovs

`LXSS_LLLL_PPPRRR_YYYYMMDD_yyyymmdd_CC_TX`

- L = Landsat
- X = Sensor (“C”=OLI/TIRS combined, “O”=OLI-only, “T”=TIRS-only, “E”=ETM+, “T”=“TM, “M”=MSS)
- SS = Satellite (”07”=Landsat 7, “08”=Landsat 8)
- LLL = Processing correction level (L1TP/L1GT/L1GS)
- PPP = WRS path
- RRR = WRS row
- YYYYMMDD = Acquisition year, month, day
- yyyymmdd - Processing year, month, day
- CC = Collection number (01, 02, …)
- TX = Collection category (“RT”=Real-Time, “T1”=Tier 1, “T2”=Tier 2)


In [ ]:
for f in file_list:
    fovs = gpd.read_file(f)
    for i in range(len(fovs)):
        
        fov = fovs.iloc[i]

        bounds = fov.geometry.bounds
        bbox, footprint = get_bbox_and_footprint(bounds)

        # Project to WGS84 to obtain in geometric coordinates - TODO: IF NEEDED
        geo_bounds = transform_bounds(fovs.crs.to_string(), 'EPSG:4326', *bbox)
        geo_footprint = transform_geom(fovs.crs.to_string(), 'EPSG:4326', footprint)

        dt = datetime.fromisoformat(fov.time)
        dtz = dt.astimezone(timezone.utc)

        if fov.satellite == "LANDSAT 8":
            id = str("LC08_L0XX_" + dtz.strftime('%Y%m%d%M%S'))
        if fov.satellite == "LANDSAT 9":
            id = str("LC09_L0XX_" + dtz.strftime('%Y%m%d%M%S'))

        item = pystac.Item(
            # id=str(i),
            id = id,
            geometry = geo_footprint,
            bbox = geo_bounds,
            datetime = dtz,
            stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json'],
            properties=dict(
            # tile=tile
            )
        )

        catalog.add_item(item)

catalog.normalize_hrefs('./stac_catalog')
catalog.save()

# print(len(list(catalog.get_items())))
# catalog.describe()

In [ ]:
from pystac import Catalog, get_stac_version
root_catalog = Catalog.from_file('./stac_catalog/catalog.json')
print(f"ID: {root_catalog.id}")
print(f"Title: {root_catalog.title or 'N/A'}")
print(f"Description: {root_catalog.description or 'N/A'}")

In [ ]:
item = root_catalog.get_item("LC09_L0XX_202206245948", recursive=True)


In [ ]:
item.geometry